# Create a SageMaker MLOps Project for Pipelines
Note:  This requires that you have enabled products within SageMaker Studio

![](../img/enable-service-catalog-portfolio-for-studio.png)

In [1]:
import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd
from pprint import pprint

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
sc = boto3.Session().client(service_name='servicecatalog', region_name=region)
sts = boto3.Session().client(service_name='sts', region_name=region)
iam = boto3.Session().client(service_name='iam', region_name=region)

In [2]:
search_response = sc.search_products(
   Filters={
       'FullTextSearch': 
       [
           'MLOps template for model building, training, and deployment'
       ]
   }
)

sagemaker_pipeline_product_id = search_response['ProductViewSummaries'][0]['ProductId']
print(sagemaker_pipeline_product_id)

# pprint(search_response)

prod-j3ufw6hl7utxm


In [3]:
describe_response = sc.describe_product(Id=sagemaker_pipeline_product_id)

sagemaker_pipeline_product_provisioning_artifact_id = describe_response['ProvisioningArtifacts'][0]['Id']

pprint(describe_response)

{'Budgets': [],
 'LaunchPaths': [{'Id': 'lpv2-otfcrq7zdg5sc',
                  'Name': 'Amazon SageMaker Solutions and ML Ops products'}],
 'ProductViewSummary': {'HasDefaultPath': False,
                        'Id': 'prodview-wbmrovteqfhoy',
                        'Name': 'MLOps template for model building, training, '
                                'and deployment',
                        'Owner': 'Amazon SageMaker',
                        'ProductId': 'prod-j3ufw6hl7utxm',
                        'ShortDescription': 'This template enables you to '
                                            'easily build, train, and deploy '
                                            'machine learning models. You can '
                                            'adopt MLOps best practices and '
                                            'enable Continuous '
                                            'Integration/Continuous Deployment '
                                            'for build

In [4]:
print(sagemaker_pipeline_product_provisioning_artifact_id)

pa-oacphmo7m2bji


# Create a SageMaker Project


In [5]:
import time

timestamp = int(time.time())

In [6]:
sagemaker_project_name = 'dsoaws-{}'.format(timestamp)

create_response = sm.create_project(
    ProjectName=sagemaker_project_name,
    ProjectDescription='dsoaws-{}'.format(timestamp),
    ServiceCatalogProvisioningDetails={
        'ProductId': sagemaker_pipeline_product_id,
        'ProvisioningArtifactId': sagemaker_pipeline_product_provisioning_artifact_id
     }
)

sagemaker_project_id = create_response['ProjectId']
sagemaker_project_arn = create_response['ProjectArn']

pprint(create_response)

{'ProjectArn': 'arn:aws:sagemaker:us-east-1:835319576252:project/dsoaws-1609835656',
 'ProjectId': 'p-ef3uutdiztnk',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '112',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Tue, 05 Jan 2021 08:34:15 GMT',
                                      'x-amzn-requestid': 'adfdac25-8eb1-433e-9ac7-55f7326970e3'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'adfdac25-8eb1-433e-9ac7-55f7326970e3',
                      'RetryAttempts': 0}}


In [7]:
sagemaker_project_name_and_id = '{}-{}'.format(sagemaker_project_name, sagemaker_project_id)

print(sagemaker_project_name_and_id)

dsoaws-1609835656-p-ef3uutdiztnk


# _Wait for the Project to be Created_

In [8]:
%%time

import time

try:
    create_project_response = sm.describe_project(ProjectName=sagemaker_project_name)
    project_status = create_project_response['ProjectStatus']
    print('Creating Project...')

    while project_status in ['Pending', 'CreateInProgress']:
        print('Please wait...')
        time.sleep(30)
        create_project_response = sm.describe_project(ProjectName=sagemaker_project_name)
        project_status = create_project_response['ProjectStatus']
        print('Project status: {}'.format(project_status))

    if project_status == 'CreateCompleted':   
        print('Project {}'.format(project_status))

    else:
        print('Project status: {}'.format(project_status))
        raise Exception('Project not created.')
        
except Exception as e:
    print(e)
    
print(create_project_response)

Creating Project...
Please wait...
Project status: CreateInProgress
Please wait...
Project status: CreateInProgress
Please wait...
Project status: CreateInProgress
Please wait...
Project status: CreateInProgress
Please wait...
Project status: CreateCompleted
Project CreateCompleted
{'ProjectArn': 'arn:aws:sagemaker:us-east-1:835319576252:project/dsoaws-1609835656', 'ProjectName': 'dsoaws-1609835656', 'ProjectId': 'p-ef3uutdiztnk', 'ProjectDescription': 'dsoaws-1609835656', 'ServiceCatalogProvisioningDetails': {'ProductId': 'prod-j3ufw6hl7utxm', 'ProvisioningArtifactId': 'pa-oacphmo7m2bji'}, 'ServiceCatalogProvisionedProductDetails': {'ProvisionedProductId': 'pp-w6pysji7lvncm'}, 'ProjectStatus': 'CreateCompleted', 'CreationTime': datetime.datetime(2021, 1, 5, 8, 34, 16, 349000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'd568e4ef-692a-4bdb-868a-3e8e97603bd3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd568e4ef-692a-4bdb-868a-3e8e97603bd3', 'content-type': 'app

# _Wait for Project to be Created ^^ Above ^^_

# Attach IAM Policies for FeatureStore 
This is used for Code Build Pipeline Executions.

In [9]:
sc_role_name='AmazonSageMakerServiceCatalogProductsUseRole'

In [10]:
account_id = sts.get_caller_identity()['Account']
print(account_id)

835319576252


In [11]:
# sc_role_arn = 'arn:aws:iam::{}:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole'.format(account_id)
# print(sc_role_arn)

In [12]:
response = iam.attach_role_policy(
    RoleName=sc_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
)

print(response)

{'ResponseMetadata': {'RequestId': '36827aa9-9dce-4011-b755-dea2156b4a1c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '36827aa9-9dce-4011-b755-dea2156b4a1c', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Tue, 05 Jan 2021 08:36:46 GMT'}, 'RetryAttempts': 0}}


In [13]:
response = iam.attach_role_policy(
    RoleName=sc_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFeatureStoreAccess'
)

print(response)

{'ResponseMetadata': {'RequestId': '2e3c0cae-ab32-4b3c-aef4-75b79f194f2a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2e3c0cae-ab32-4b3c-aef4-75b79f194f2a', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Tue, 05 Jan 2021 08:36:47 GMT'}, 'RetryAttempts': 0}}


# Stop the `Abalone` Sample Pipeline that Ships with SageMaker Pipelines
The sample "abalone" pipeline starts automatically when we create the project.  We want to stop this pipeline to release these resources and use them for our own pipeline.

In [14]:
sample_abalone_pipeline_execution_arn = sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)['PipelineExecutionSummaries'][0]['PipelineExecutionArn']

print(sample_abalone_pipeline_execution_arn)

arn:aws:sagemaker:us-east-1:835319576252:pipeline/dsoaws-1609835656-p-ef3uutdiztnk/execution/q6xwg6lbgpz0


In [15]:
sm.stop_pipeline_execution(PipelineExecutionArn=sample_abalone_pipeline_execution_arn)

{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/dsoaws-1609835656-p-ef3uutdiztnk/execution/q6xwg6lbgpz0',
 'ResponseMetadata': {'RequestId': 'bb7fe512-89b9-454a-a309-53bef5aa241b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bb7fe512-89b9-454a-a309-53bef5aa241b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '132',
   'date': 'Tue, 05 Jan 2021 08:36:47 GMT'},
  'RetryAttempts': 0}}

In [16]:
%%time

try:
    describe_pipeline_execution_response = sm.describe_pipeline_execution(PipelineExecutionArn=sample_abalone_pipeline_execution_arn)
    pipeline_execution_status = describe_pipeline_execution_response['PipelineExecutionStatus']

    while pipeline_execution_status not in ['Stopped', 'Failed']:
        print('Please wait...')
        time.sleep(30)
        describe_pipeline_execution_response = sm.describe_pipeline_execution(PipelineExecutionArn=sample_abalone_pipeline_execution_arn)
        pipeline_execution_status = describe_pipeline_execution_response['PipelineExecutionStatus']
        print('Pipeline execution status: {}'.format(pipeline_execution_status))

    if pipeline_execution_status in ['Stopped', 'Failed']:   
        print('Pipeline execution status {}'.format(pipeline_execution_status))
    else:
        print('Pipeline execution status: {}'.format(pipeline_execution_status))
        raise Exception('Pipeline execution not deleted.')
        
except Exception as e:
    print(e)
    
print(describe_pipeline_execution_response)

Please wait...
Pipeline execution status: Stopping
Please wait...
Pipeline execution status: Stopping
Please wait...
Pipeline execution status: Stopping
Please wait...
Pipeline execution status: Stopping
Please wait...
Pipeline execution status: Stopping
Please wait...
Pipeline execution status: Stopped
Pipeline execution status Stopped
{'PipelineArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/dsoaws-1609835656-p-ef3uutdiztnk', 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/dsoaws-1609835656-p-ef3uutdiztnk/execution/q6xwg6lbgpz0', 'PipelineExecutionDisplayName': 'execution-1609835758114', 'PipelineExecutionStatus': 'Stopped', 'CreationTime': datetime.datetime(2021, 1, 5, 8, 35, 58, 44000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2021, 1, 5, 8, 39, 38, 838000, tzinfo=tzlocal()), 'CreatedBy': {}, 'LastModifiedBy': {}, 'ResponseMetadata': {'RequestId': '5057a6af-4c35-47bb-9c8c-3afdd705a5e7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz

In [17]:
sm.delete_pipeline(PipelineName=sagemaker_project_name_and_id)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/dsoaws-1609835656-p-ef3uutdiztnk',
 'ResponseMetadata': {'RequestId': 'c4278191-75a2-47e7-8141-77c51b4085fe',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c4278191-75a2-47e7-8141-77c51b4085fe',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Tue, 05 Jan 2021 08:39:47 GMT'},
  'RetryAttempts': 0}}

# Clone the MLOps Repositories in AWS CodeCommit

In [18]:
import os

sm_studio_root_path='/root/' 
sm_notebooks_root_path='/home/ec2-user/SageMaker/'

root_path = sm_notebooks_root_path if os.path.isdir(sm_notebooks_root_path) else sm_studio_root_path

print(root_path)

/home/ec2-user/SageMaker/


In [19]:
print(region)

us-east-1


In [20]:
code_commit_repo1 = 'https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-{}-modelbuild'.format(region, sagemaker_project_name_and_id)
print(code_commit_repo1)

https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-dsoaws-1609835656-p-ef3uutdiztnk-modelbuild


In [21]:
sagemaker_mlops_build_code = '{}{}/sagemaker-{}-modelbuild'.format(root_path, sagemaker_project_name_and_id, sagemaker_project_name_and_id)
print(sagemaker_mlops_build_code)

/home/ec2-user/SageMaker/dsoaws-1609835656-p-ef3uutdiztnk/sagemaker-dsoaws-1609835656-p-ef3uutdiztnk-modelbuild


In [22]:
code_commit_repo2 = 'https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-{}-modeldeploy'.format(region, sagemaker_project_name_and_id)
print(code_commit_repo2)

https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-dsoaws-1609835656-p-ef3uutdiztnk-modeldeploy


In [23]:
sagemaker_mlops_deploy_code = '{}{}/sagemaker-{}-modeldeploy'.format(root_path, sagemaker_project_name_and_id, sagemaker_project_name_and_id)
print(sagemaker_mlops_deploy_code)

/home/ec2-user/SageMaker/dsoaws-1609835656-p-ef3uutdiztnk/sagemaker-dsoaws-1609835656-p-ef3uutdiztnk-modeldeploy


In [24]:
!git config --global credential.helper '!aws codecommit credential-helper $@'
!git config --global credential.UseHttpPath true

# _Wait for Project to be Created ^^ Above ^^_

In [25]:
!git clone $code_commit_repo1 $sagemaker_mlops_build_code

Cloning into '/home/ec2-user/SageMaker/dsoaws-1609835656-p-ef3uutdiztnk/sagemaker-dsoaws-1609835656-p-ef3uutdiztnk-modelbuild'...
remote: Counting objects: 26, done.
Unpacking objects: 100% (26/26), done.


In [26]:
!git clone $code_commit_repo2 $sagemaker_mlops_deploy_code

Cloning into '/home/ec2-user/SageMaker/dsoaws-1609835656-p-ef3uutdiztnk/sagemaker-dsoaws-1609835656-p-ef3uutdiztnk-modeldeploy'...
remote: Counting objects: 12, done.
Unpacking objects: 100% (12/12), done.


# Remove Stock `Abalone` Example Code

In [27]:
!rm -rf $sagemaker_mlops_build_code/pipelines/abalone

# Copy Workshop Code Into Local Project Folders

In [28]:
workshop_project_build_code='{}workshop/10_pipeline/sagemaker/sagemaker-project-modelbuild'.format(root_path)
print(workshop_project_build_code)

/home/ec2-user/SageMaker/workshop/10_pipeline/sagemaker/sagemaker-project-modelbuild


In [29]:
workshop_project_deploy_code='{}workshop/10_pipeline/sagemaker/sagemaker-project-modeldeploy'.format(root_path)
print(workshop_project_deploy_code)

/home/ec2-user/SageMaker/workshop/10_pipeline/sagemaker/sagemaker-project-modeldeploy


In [30]:
!cp -R $workshop_project_build_code/* $sagemaker_mlops_build_code/

In [31]:
!cp -R $workshop_project_deploy_code/* $sagemaker_mlops_deploy_code/

# Commit New Code 

In [32]:
print(sagemaker_mlops_build_code)

/home/ec2-user/SageMaker/dsoaws-1609835656-p-ef3uutdiztnk/sagemaker-dsoaws-1609835656-p-ef3uutdiztnk-modelbuild


In [33]:
!cd $sagemaker_mlops_build_code; git status; git add --all .; git commit -m "Data Science on AWS"; git push

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   README.md
	modified:   codebuild-buildspec.yml
	deleted:    pipelines/abalone/__init__.py
	deleted:    pipelines/abalone/evaluate.py
	deleted:    pipelines/abalone/pipeline.py
	deleted:    pipelines/abalone/preprocess.py
	modified:   pipelines/run_pipeline.py
	modified:   setup.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	pipelines/dsoaws/

no changes added to commit (use "git add" and/or "git commit -a")
[master a09e278] Data Science on AWS
 14 files changed, 2229 insertions(+), 452 deletions(-)
 delete mode 100644 pipelines/abalone/evaluate.py
 delete mode 100644 pipelines/abalone/pipeline.py
 delete mode 100644 pipelines/abalone/preprocess.py
 rename pipelines/{abalone => dsoaws}/__init__.py (1

In [34]:
!cd $sagemaker_mlops_deploy_code; git status; git add --all .; git commit -m "Data Science on AWS"; git push

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   README.md
	modified:   prod-config.json
	modified:   staging-config.json

no changes added to commit (use "git add" and/or "git commit -a")
[master a17e050] Data Science on AWS
 3 files changed, 3 insertions(+), 3 deletions(-)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 442 bytes | 442.00 KiB/s, done.
Total 5 (delta 4), reused 0 (delta 0)
To https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-dsoaws-1609835656-p-ef3uutdiztnk-modeldeploy
   8518ca8..a17e050  master -> master


# Store the Variables

In [35]:
%store sagemaker_mlops_build_code
%store sagemaker_mlops_deploy_code
%store sagemaker_project_name
%store sagemaker_project_id
%store sagemaker_project_name_and_id
%store sagemaker_project_arn
%store sagemaker_pipeline_product_id
%store sagemaker_pipeline_product_provisioning_artifact_id

Stored 'sagemaker_mlops_build_code' (str)
Stored 'sagemaker_mlops_deploy_code' (str)
Stored 'sagemaker_project_name' (str)
Stored 'sagemaker_project_id' (str)
Stored 'sagemaker_project_name_and_id' (str)
Stored 'sagemaker_project_arn' (str)
Stored 'sagemaker_pipeline_product_id' (str)
Stored 'sagemaker_pipeline_product_provisioning_artifact_id' (str)


In [36]:
!ls -al $sagemaker_mlops_build_code/pipelines/dsoaws/

total 108
drwxrwxr-x 4 ec2-user ec2-user  4096 Jan  5 08:39 .
drwxrwxr-x 4 ec2-user ec2-user  4096 Jan  5 08:39 ..
-rw-rw-r-- 1 ec2-user ec2-user  8923 Jan  5 08:39 evaluate_model_metrics.py
-rw-rw-r-- 1 ec2-user ec2-user  3042 Jan  5 08:39 inference.py
-rw-rw-r-- 1 ec2-user ec2-user     0 Jan  5 08:39 __init__.py
-rw-rw-r-- 1 ec2-user ec2-user 15072 Jan  5 08:39 pipeline.py
-rw-rw-r-- 1 ec2-user ec2-user 26680 Jan  5 08:39 preprocess-scikit-text-to-bert-feature-store.py
drwxrwxr-x 2 ec2-user ec2-user  4096 Jan  5 08:39 __pycache__
drwxrwxr-x 2 ec2-user ec2-user  4096 Jan  5 08:39 test_data
-rw-rw-r-- 1 ec2-user ec2-user 28793 Jan  5 08:39 tf_bert_reviews.py


In [37]:
!pygmentize $sagemaker_mlops_build_code/pipelines/dsoaws/pipeline.py

"""
Example workflow pipeline script for BERT pipeline.

                                                 . -RegisterModel
                                                .
    Process-> Train -> (Evaluate -> Condition) .
                                                .
                                                 . -(stop)

Implements a get_pipeline(**kwargs) method.
"""

import os
import boto3
import logging
import time

from botocore.exceptions import ClientError

import sagemaker
import sagemaker.session
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.tensorflow import TensorFlow

from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)

from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat
)

from

# Wait for Pipeline Execution to Start
Now that we have committed code, our pipeline will start.  Let's wait for the pipeline to start.


In [38]:
%%time

import time
from pprint import pprint

while True:
    try:
        print('Listing executions for our pipeline...')
        list_executions_response = sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)['PipelineExecutionSummaries']
        break;
    except Exception as e:
        print('Please wait...')
        time.sleep(30)   
    #    print(e)
    
pprint(list_executions_response)

Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
[{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/dsoaws-1609835656-p-ef3uutdiztnk/execution/x6eflnsfnpps',
  'PipelineExecutionDisplayName': 'execution-1609836085954',
  'PipelineExecutionStatus': 'Executing',
  'StartTime': datetime.datetime(2021, 1, 5, 8, 41, 25, 795000, tzinfo=tzlocal())}]
CPU times: user 53.2 ms, sys: 0 ns, total: 53.2 ms
Wall time: 1min 30s


In [39]:
%store

Stored variables and their in-db values:
balance_dataset                                                 -> True
experiment_name                                                 -> 'Amazon-Customer-Reviews-BERT-Experiment-160964927
feature_group_name                                              -> 'reviews-feature-group-1609649274'
feature_store_offline_prefix                                    -> 'reviews-feature-store-1609649274'
ingest_create_athena_db_passed                                  -> True
ingest_create_athena_table_tsv_passed                           -> True
max_seq_length                                                  -> 64
processed_test_data_s3_uri                                      -> 's3://sagemaker-us-east-1-835319576252/sagemaker-s
processed_train_data_s3_uri                                     -> 's3://sagemaker-us-east-1-835319576252/sagemaker-s
processed_validation_data_s3_uri                                -> 's3://sagemaker-us-east-1-835319576252/sagemaker